# AML HOMEWORK 4
## Hima Bindu (hb2635), Swarna Bharathi Mantena (sm4776)

# TASK 2

In [1]:
import pandas as pd

# Read in data COLAB
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd

dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/aml/hw4/winemag-data-130k-v2.csv')
dataset.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## Preprocessing steps: 
- Preprocessing was done as discussed in Task 1. 


In [0]:
stats = dataset['price'].describe()

upper_quartile = stats[6]
lower_quartile = stats[4]

iqr = upper_quartile - lower_quartile
upper_whisker = dataset['price'][dataset.price<=upper_quartile+1.5*iqr].max()
price_wo_outliers = dataset['price'][dataset.price<=upper_quartile+1.5*iqr]

dataset_updated = dataset[dataset.price<=upper_quartile+1.5*iqr]

In [4]:
print(len(dataset_updated))
print(len(dataset))

113734
129971


In [0]:
X = dataset_updated.drop(["Unnamed: 0", "taster_twitter_handle", "points"], axis = 1)
X['description+title'] = X['description'] + X['title']
y = dataset_updated["points"]

In [6]:
!pip install --upgrade category_encoders

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.1.0)


In [0]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from category_encoders import TargetEncoder

X_train, X_test, y_train, y_test = train_test_split(X, y)
#imputation
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X], index=X.columns)
        return self

    def transform(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X], index=X.columns)
        return X.fillna(self.fill)

X_train = DataFrameImputer().fit_transform(X_train)
X_test = DataFrameImputer().transform(X_test)

In [8]:
import spacy
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg", disable = ["taggle","parser","ner"])

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [9]:
desc_train = [nlp(d).vector for d in X_train['description']]
title_train = [nlp(d).vector for d in X_train['title']]
desc = np.array(desc_train)
title = np.array(title_train)
combined_X_train = np.concatenate((desc, title), axis=1)
combined_X_train.shape

(85300, 600)

In [10]:
from sklearn.linear_model import Ridge

model_rr = Ridge(alpha=1)
scores_rr = cross_val_score(model_rr, combined_X_train, y_train)

np.mean(scores_rr)

0.5643502983037137

The R^2 value of the Word2Vec model seems to have reduced compared to the the best R^2 value observed in Taks 1. This might be because the Word2Vec model is trained on a different dataset that might not have necessarily captured the context that appears in this data properly. 

In [11]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()
scores_lr = cross_val_score(model_lr, combined_X_train, y_train)

np.mean(scores_lr)

0.564357873751071

In [52]:
combined_X_df = pd.DataFrame(combined_X_train)
# combined_X_df.reset_index(drop=True, inplace=True)
combined_X_df_noindex = combined_X_df.reset_index(drop=True, inplace = False)
col_indices = [str(d) for d in range(600)]
combined_X_df_noindex.columns = col_indices

# X_train.reset_index(drop=True, inplace=True)
X_train_noindex = X_train.reset_index(drop=True, inplace = False)
X_train_dropped = X_train_noindex.drop(['description', 'title'], axis=1) 
X_test_dropped = X_test.drop(['description', 'title'], axis=1) 
X_train_final = pd.concat([X_train_dropped, combined_X_df_noindex], axis=1)
X_train_final.index =  X_train.index

X_train_final.shape

(85300, 610)

In [56]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer

cat_feature = ['country', 'designation', 'province',
'region_1', 'region_2', 'taster_name','variety', 'winery']
text_feature = ['description','title']
cont_feature = ['price']
cont_f = ['price'] + col_indices

preprocess = make_column_transformer(('passthrough', cont_f), (TargetEncoder(), ['designation', 'province', 'region_1', 'variety', 'winery']),
(OneHotEncoder(handle_unknown = 'ignore'), ['country','region_2','taster_name']),(CountVectorizer(ngram_range=(1, 2)),'description+title'))

model_rr = make_pipeline(preprocess, Ridge(alpha = 10))
scores_rr = cross_val_score(model_rr, X_train_final, y_train, cv = 5)

np.mean(scores_rr)

0.7768605219608047

We observe that the R^2 value for the combined model (Word2Vec and BoW) is almost the same as initial BoW + Linear model. 